# Group Project: University Course Timetabling Problem (Extended Model)

Course Title: **DAMO-610-3: Winter 2025 Operations Analytics**  
Professor Name: **Professor Hany Osman**  
Submission Date: **March 23, 2025**  
Submitted By:  
1. **Catherine C. Calantoc (NF1001422)**  
2. **Dev D. Rabadia (NF1005560)**  
3. **Miko L. Tan (NF1008647)**  
4. **Rosario D. Torres (NF1001385)**  

**Class Scheduling and Instructor Assignment in Higher Education:** In higher education, class scheduling and instructor assignment is a multi-faceted problem requiring the balancing of faculty availability, classroom capacity, student preferences, and the curriculum. Inefficient scheduling results in wasted resources, conflicts, and dissatisfaction among students and instructors. Integer programming optimization models enable universities to automate and optimize their timetables to schedule courses appropriately while balancing instructor workload, maximizing student accessibility, and minimizing operational cost. An optimal scheduling system provides the backbone for academic planning, reducing administrative hassle and improving the learning experience for students.

**This extended model enhances practicality by ensuring consistent scheduling patterns for multi-session courses (Monday-Wednesday-Friday or Tuesday-Thursday), universal 30-minute breaks for building transfers within the same location, optional lecturer-specific breaks, workload limits, single-location assignments per day, and contingency room reservations. The objective is to maximize lecturer and course preferences while maintaining operational feasibility and reserving capacity for contingencies.**

**Indices:**  
- ***L***: Set of lecturers.  
- ***CS***: Set of course sections.  
- ***R***: Set of rooms.  
- ***D***: Set of days (Monday, Tuesday, Wednesday, Thursday, Friday).  
- ***T***: Set of time periods (30-minute increments).  
- ***P***: Set of day patterns (0 for MWF, 1 for TTh).  
- ***LOC***: Set of locations (e.g., UNF NF, UNF Fort Erie).  

**Parameters:**
- ***FreqD[cs]***: Frequency of course section cs per week (1 or >1).  
- ***DurT[cs]***: Duration of course section cs in time units (30-minute increments).  
- ***MinC[l]***: Minimum number of courses lecturer l must teach.  
- ***MaxC[l]***: Maximum number of courses lecturer l can teach.  
- ***MaxT[l]***: Maximum teaching hours per week for lecturer l.  
- ***BreakT[l]***: Binary indicator (0 or 1) for whether lecturer l requires a 30-minute break between all sessions (optional, separate from building transfers).
- ***p1[l, cs]***: Preference score for lecturer l teaching course section cs.  
- ***p2[l, d]***: Preference score for lecturer l teaching on day d.  
- ***p3[cs, d]***: Preference score for course section cs being scheduled on day d.  
- ***p4[l, loc]***: Preference score for lecturer l teaching at location loc.  
- ***p5[cs, r]***: Preference score for course section cs being held in room r.  
- ***p6[r]***: Room condition score for room r.  
- ***a1[l, cs]***: Binary indicator if lecturer l is qualified to teach course section cs.  
- ***a4[cs, r]***: Binary indicator if room r meets the type requirement for course section cs.  
- ***a5[cs, r]***: Binary indicator if room r meets the PWD requirement for course section cs.  
- ***a6[r]***: Binary indicator if room r is a contingency room.  
- ***a7[r]***: Binary indicator if room r is available (not closed for renovation).  
- ***MWF_Days***: Subset of days {Monday, Wednesday, Friday}.  
- ***TTh_Days***: Subset of days {Tuesday, Thursday}.  
- ***Building[r]***: Building identifier for room r (e.g., UNF NF Building 1).  
- ***Location[r]***: Location identifier for room r (e.g., UNF NF).  
- ***ReservePct***: Percentage of total room-time capacity to reserve for contingencies (e.g., 0.1 for 10%).  

**Constraints:**  

1. **Course Section Frequency and Day Patterns:** Ensures each course section is scheduled according to its frequency, with multi-session courses (`FreqD[cs] > 1`) following either MWF or TTh patterns.  
     $$  
     \sum_{l \in L, d \in D, r \in R, t \in T} Z[l, cs, d, r, t] = FreqD[cs] \quad \forall cs \in CS
       $$
     For $ FreqD[cs] > 1 $:
     $$  
     \sum_{p \in P} Pattern[cs, p] = 1 \quad \forall cs \in CS
       $$
     $$  
     \sum_{l \in L, d \in MWF_Days, r \in R, t \in T} Z[l, cs, d, r, t] = FreqD[cs] \cdot Pattern[cs, 0] \quad \forall cs \in CS
       $$
     $$  
     \sum_{l \in L, d \in TTh_Days, r \in R, t \in T} Z[l, cs, d, r, t] = FreqD[cs] \cdot Pattern[cs, 1] \quad \forall cs \in CS
       $$

2. **Consistent Time Period:** Ensures all sessions of a course section with frequency > 1 occur at the same time period.  
     For $ FreqD[cs] > 1 $:
     $$  
     \sum_{t \in T} FixedTime[cs, t] = 1 \quad \forall cs \in CS
       $$
     $$  
     Z[l, cs, d, r, t] \leq FixedTime[cs, t] \quad \forall l \in L, cs \in CS, d \in D, r \in R, t \in T
       $$

3. **No Lecturer Overlap:** Prevents a lecturer from being scheduled for multiple sessions at the same time.  
     $$  
     \sum_{cs \in CS, r \in R} Z[l, cs, d, r, t] \leq 1 \quad \forall l \in L, d \in D, t \in T
       $$

4. **Lecturer-Course Feasibility:** Ensures lecturers only teach course sections they are qualified for.  
     $$  
     \sum_{d \in D, r \in R, t \in T} Z[l, cs, d, r, t] \leq a1[l, cs] \cdot FreqD[cs] \quad \forall l \in L, cs \in CS
       $$

5. **No Room Overlap:** Ensures a room is not booked for multiple sessions at the same time.  
     $$  
     \sum_{l \in L, cs \in CS} Z[l, cs, d, r, t] \leq 1 \quad \forall r \in R, d \in D, t \in T
       $$

6. **Lecturer Availability:** Limits the total teaching hours per week for each lecturer.  
     $$  
     \sum_{cs \in CS, d \in D, r \in R, t \in T} Z[l, cs, d, r, t] \cdot DurT[cs] \leq MaxT[l] \quad \forall l \in L
       $$

7. **Course-Room Fit:** Ensures rooms meet course section requirements (type, PWD access, availability).  
     $$  
     \sum_{l \in L, d \in D, t \in T} Z[l, cs, d, r, t] \leq a4[cs, r] \cdot a5[cs, r] \cdot a7[r] \quad \forall cs \in CS, r \in R
       $$

8. **Lecturer Breaks:** Ensures a 30-minute break between all sessions for lecturers with `BreakT[l] = 1` (optional, not tied to building transfers).  
     $$  
     Z[l, cs_1, d, r_1, t_1] + Z[l, cs_2, d, r_2, t_2] \leq 1 \quad \forall l \in L \text{ where } BreakT[l] = 1, cs_1, cs_2 \in CS, d \in D, r_1, r_2 \in R, t_1, t_2 \in T
       $$
     Where $ 0 < |t_2 - t_1| < 2 $ (30 minutes = 1 time period, check up to 2 periods).

9. **Workload Limits:** Ensures the number of courses assigned to each lecturer is within their minimum and maximum limits.  
     $$  
     MinC[l] \leq \sum_{cs \in CS} teaches[l, cs] \leq MaxC[l] \quad \forall l \in L
       $$
     Where:
     $$  
     teaches[l, cs] \geq \sum_{d \in D, r \in R, t \in T} Z[l, cs, d, r, t] / FreqD[cs] \quad \forall l \in L, cs \in CS
       $$
     $$  
     teaches[l, cs] \leq \sum_{d \in D, r \in R, t \in T} Z[l, cs, d, r, t] \quad \forall l \in L, cs \in CS
       $$

10. **Lecturer Location per Day:** Ensures each lecturer is assigned to only one location per day.  
      $$  
      \sum_{loc \in LOC} uses_location[l, d, loc] = 1 \quad \forall l \in L, d \in D
        $$
      Where:
      $$  
      loc_usage[l, d, loc] = \sum_{r \in R_{loc}, cs \in CS, t \in T} Z[l, cs, d, r, t] \quad \forall l \in L, d \in D, loc \in LOC
        $$
      $$  
      uses_location[l, d, loc] \cdot (|CS| \cdot |T|) \geq loc_usage[l, d, loc] \quad \forall l \in L, d \in D, loc \in LOC
        $$

11. **Contingency Reservation:** Ensures a percentage of total room-time capacity remains unscheduled for contingencies.  
      $$  
      \sum_{l \in L, cs \in CS, d \in D, r \in R, t \in T} Z[l, cs, d, r, t] \leq (1 - ReservePct) \cdot |R| \cdot |D| \cdot |T|
        $$

12. **Building Transfer Break:** Ensures a 30-minute break between sessions in different buildings within the same location for all lecturers.  
      $$  
      Z[l, cs_1, d, r_1, t_1] + Z[l, cs_2, d, r_2, t_2] \leq 1 \quad \forall l \in L, cs_1, cs_2 \in CS, d \in D, r_1, r_2 \in R, t_1, t_2 \in T
        $$
      Where:
      - $ Location[r_1] = Location[r_2] $ (same location),
      - $ Building[r_1] \neq Building[r_2] $ (different buildings),
      - $ 0 < |t_2 - t_1| < 2 $ (30-minute gap).

**Decision Variables:**

- **Z[l, cs, d, r, t]**: Binary variable indicating whether lecturer $ l \in L $ teaches course section $ cs \in CS $ on day $ d \in D $ in room $ r \in R $ at time period $ t \in T $.  
  - $ Z[l, cs, d, r, t] = 1 $ if assigned, $ 0 $ otherwise.  
- **Pattern[cs, p]**: Binary variable indicating the day pattern for course section $ cs \in CS $ with frequency > 1, where $ p \in P $ (0 for MWF, 1 for TTh).  
  - $ Pattern[cs, p] = 1 $ if pattern $ p $ is selected, $ 0 $ otherwise.  
- **FixedTime[cs, t]**: Binary variable indicating the fixed time period for course section $ cs \in CS $ with frequency > 1, where $ t \in T $.  
  - $ FixedTime[cs, t] = 1 $ if time $ t $ is selected, $ 0 $ otherwise.  
- **teaches[l, cs]**: Binary variable indicating whether lecturer $ l \in L $ teaches course section $ cs \in CS $.  
  - $ teaches[l, cs] = 1 $ if assigned, $ 0 $ otherwise.  
- **uses_location[l, d, loc]**: Binary variable indicating whether lecturer $ l \in L $ is assigned to location $ loc \in LOC $ on day $ d \in D $.  
  - $ uses_location[l, d, loc] = 1 $ if assigned, $ 0 $ otherwise.  
  
**Objective Function:**  

- **Maximize Preferences and Minimize Penalties:** Maximizes preferences for lecturer assignments and room conditions while minimizing the use of contingency rooms.  
    $$  
    \text{Maximize} \sum_{l \in L, cs \in CS, d \in D, r \in R, t \in T} Z[l, cs, d, r, t] \cdot (p1[l, cs] + p2[l, d] + p3[cs, d] + p4[l, Location[r]] + p5[cs, r] + p6[r] - \beta \cdot a6[r])
      $$
    Where $ \beta $ is a penalty coefficient (e.g., 1).

**Subject To:**

1. *Course Section Frequency and Day Patterns*  
   $$  
   \sum_{l \in L, d \in D, r \in R, t \in T} Z[l, cs, d, r, t] = FreqD[cs] \quad \forall cs \in CS
     $$
   For $ FreqD[cs] > 1 $:
   $$  
   \sum_{p \in P} Pattern[cs, p] = 1 \quad \forall cs \in CS
     $$
   $$  
   \sum_{l \in L, d \in MWF_Days, r \in R, t \in T} Z[l, cs, d, r, t] = FreqD[cs] \cdot Pattern[cs, 0] \quad \forall cs \in CS
     $$
   $$  
   \sum_{l \in L, d \in TTh_Days, r \in R, t \in T} Z[l, cs, d, r, t] = FreqD[cs] \cdot Pattern[cs, 1] \quad \forall cs \in CS
     $$

2. *Consistent Time Period*  
   For $ FreqD[cs] > 1 $:
   $$  
   \sum_{t \in T} FixedTime[cs, t] = 1 \quad \forall cs \in CS
     $$
   $$  
   Z[l, cs, d, r, t] \leq FixedTime[cs, t] \quad \forall l \in L, cs \in CS, d \in D, r \in R, t \in T
     $$

3. *No Lecturer Overlap*  
   $$  
   \sum_{cs \in CS, r \in R} Z[l, cs, d, r, t] \leq 1 \quad \forall l \in L, d \in D, t \in T
     $$

4. *Lecturer-Course Feasibility*  
   $$  
   \sum_{d \in D, r \in R, t \in T} Z[l, cs, d, r, t] \leq a1[l, cs] \cdot FreqD[cs] \quad \forall l \in L, cs \in CS
     $$

5. *No Room Overlap*  
   $$  
   \sum_{l \in L, cs \in CS} Z[l, cs, d, r, t] \leq 1 \quad \forall r \in R, d \in D, t \in T
     $$

6. *Lecturer Availability*  
   $$  
   \sum_{cs \in CS, d \in D, r \in R, t \in T} Z[l, cs, d, r, t] \cdot DurT[cs] \leq MaxT[l] \quad \forall l \in L
     $$

7. *Course-Room Fit*  
   $$  
   \sum_{l \in L, d \in D, t \in T} Z[l, cs, d, r, t] \leq a4[cs, r] \cdot a5[cs, r] \cdot a7[r] \quad \forall cs \in CS, r \in R
     $$

8. *Lecturer Breaks*  
   $$  
   Z[l, cs_1, d, r_1, t_1] + Z[l, cs_2, d, r_2, t_2] \leq 1 \quad \forall l \in L \text{ where } BreakT[l] = 1, cs_1, cs_2 \in CS, d \in D, r_1, r_2 \in R, t_1, t_2 \in T
     $$
   Where $ 0 < |t_2 - t_1| < 2 $ (30 minutes = 1 time period, check up to 2 periods).

9. *Workload Limits*  
   $$  
   MinC[l] \leq \sum_{cs \in CS} teaches[l, cs] \leq MaxC[l] \quad \forall l \in L
     $$
   Where:
   $$  
   teaches[l, cs] \geq \sum_{d \in D, r \in R, t \in T} Z[l, cs, d, r, t] / FreqD[cs] \quad \forall l \in L, cs \in CS
     $$
   $$  
   teaches[l, cs] \leq \sum_{d \in D, r \in R, t \in T} Z[l, cs, d, r, t] \quad \forall l \in L, cs \in CS
     $$

10. *Lecturer Location per Day*  
    $$  
    \sum_{loc \in LOC} uses_location[l, d, loc] = 1 \quad \forall l \in L, d \in D
      $$
    Where:
    $$  
    loc_usage[l, d, loc] = \sum_{r \in R_{loc}, cs \in CS, t \in T} Z[l, cs, d, r, t] \quad \forall l \in L, d \in D, loc \in LOC
      $$
    $$  
    uses_location[l, d, loc] \cdot (|CS| \cdot |T|) \geq loc_usage[l, d, loc] \quad \forall l \in L, d \in D, loc \in LOC
      $$

11. *Contingency Reservation*  
    $$  
    \sum_{l \in L, cs \in CS, d \in D, r \in R, t \in T} Z[l, cs, d, r, t] \leq (1 - ReservePct) \cdot |R| \cdot |D| \cdot |T|
      $$

12. *Building Transfer Break*  
    $$  
    Z[l, cs_1, d, r_1, t_1] + Z[l, cs_2, d, r_2, t_2] \leq 1 \quad \forall l \in L, cs_1, cs_2 \in CS, d \in D, r_1, r_2 \in R, t_1, t_2 \in T
      $$
    Where:
    - $ Location[r_1] = Location[r_2] $ (same location),
    - $ Building[r_1] \neq Building[r_2] $ (different buildings),
    - $ 0 < |t_2 - t_1| < 2 $ (30-minute gap).

In [1]:
from ortools.linear_solver import pywraplp
import pandas as pd

load c:\Users\Miko\AppData\Local\Programs\Python\Python311\Lib\site-packages\ortools\.libs\zlib1.dll...
load c:\Users\Miko\AppData\Local\Programs\Python\Python311\Lib\site-packages\ortools\.libs\abseil_dll.dll...
load c:\Users\Miko\AppData\Local\Programs\Python\Python311\Lib\site-packages\ortools\.libs\utf8_validity.dll...
load c:\Users\Miko\AppData\Local\Programs\Python\Python311\Lib\site-packages\ortools\.libs\re2.dll...
load c:\Users\Miko\AppData\Local\Programs\Python\Python311\Lib\site-packages\ortools\.libs\libprotobuf.dll...
load c:\Users\Miko\AppData\Local\Programs\Python\Python311\Lib\site-packages\ortools\.libs\highs.dll...
load c:\Users\Miko\AppData\Local\Programs\Python\Python311\Lib\site-packages\ortools\.libs\ortools.dll...


## Step 1: Define the Data

In [2]:
# Read data from Excel
excel_file = 'uctp-extended-model-data-min.xlsx'
lecturers_df = pd.read_excel(excel_file, sheet_name='Lecturers')
courses_df = pd.read_excel(excel_file, sheet_name='Courses')
rooms_df = pd.read_excel(excel_file, sheet_name='Rooms')
days_df = pd.read_excel(excel_file, sheet_name='Days')
times_df = pd.read_excel(excel_file, sheet_name='Time Periods')
pref_lect_course_df = pd.read_excel(excel_file, sheet_name='Lecturer_Course')
pref_lect_day_df = pd.read_excel(excel_file, sheet_name='Lecturer_Day')
pref_lect_loc_df = pd.read_excel(excel_file, sheet_name='Lecturer_Location')
pref_course_day_df = pd.read_excel(excel_file, sheet_name='Course_Day')
pref_course_room_df = pd.read_excel(excel_file, sheet_name='Course_Room')

# Extract data
lecturers = lecturers_df['Lecturer Name'].tolist()
days = days_df['Day'].tolist()
time_periods = times_df['Time Period'].tolist()
rooms = rooms_df['Room Number'].tolist()

# Derive course_sections from Courses sheet
course_sections = []
for _, row in courses_df.iterrows():
    code = row['Course Code']
    num_sections = row['Number of Sections']
    for i in range(1, num_sections + 1):
        course_sections.append(f"{code}-{i}")

In [9]:
# Create a new Excel file and write collected data
output_file = 'uctp-extended-model-preprocessed-data.xlsx'
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    lecturers_df.to_excel(writer, sheet_name='Lecturers', index=False)
    courses_df.to_excel(writer, sheet_name='Courses', index=False)
    rooms_df.to_excel(writer, sheet_name='Rooms', index=False)
    days_df.to_excel(writer, sheet_name='Days', index=False)
    times_df.to_excel(writer, sheet_name='Time Periods', index=False)
    pref_lect_course_df.to_excel(writer, sheet_name='Lecturer_Course', index=False)
    pref_lect_day_df.to_excel(writer, sheet_name='Lecturer_Day', index=False)
    pref_lect_loc_df.to_excel(writer, sheet_name='Lecturer_Location', index=False)
    pref_course_day_df.to_excel(writer, sheet_name='Course_Day', index=False)
    pref_course_room_df.to_excel(writer, sheet_name='Course_Room', index=False)
    pd.DataFrame({'Course Sections': course_sections}).to_excel(writer, sheet_name='Course Sections', index=False)

print(f"Data successfully written to {output_file}")

Data successfully written to uctp-extended-model-preprocessed-data.xlsx


## Step 2: Define the Model

In [5]:
# Initialize solver
solver = pywraplp.Solver.CreateSolver('SCIP')
if not solver:
    print("Solver not created!")
    exit()

In [6]:
# Parameters
freq_d = {f"{row['Course Code']}-{i}": row['Frequency'] for _, row in courses_df.iterrows() 
          for i in range(1, row['Number of Sections'] + 1)}
dur_t = {f"{row['Course Code']}-{i}": row['Duration'] // row['Frequency'] for _, row in courses_df.iterrows() 
         for i in range(1, row['Number of Sections'] + 1)}
max_t = lecturers_df.set_index('Lecturer Name')['Max Hours Per Week'].to_dict()
min_c = lecturers_df.set_index('Lecturer Name')['Min Courses'].to_dict()
max_c = lecturers_df.set_index('Lecturer Name')['Max Courses'].to_dict()
break_t = lecturers_df.set_index('Lecturer Name')['Break Preference'].to_dict()
a1 = {(l, cs): 1 for l in lecturers for cs in course_sections}
a4 = {(cs, r): 1 for cs in course_sections for r in rooms}
a5 = {}
for cs in course_sections:
    course_code = cs.split('-')[0]
    course_row = courses_df[courses_df['Course Code'] == course_code]
    pwd_req = course_row['PWD Requirement'].iloc[0] if not course_row.empty else 'No'
    for r in rooms:
        pwd_access = rooms_df[rooms_df['Room Number'] == r]['PWD Accessible'].iloc[0]
        a5[cs, r] = 1 if pwd_req == 'No' or pwd_access == 'Yes' else 0
a7 = {r: 1 for r in rooms}
room_building = rooms_df.set_index('Room Number')['Building'].to_dict()
room_location = rooms_df.set_index('Room Number')['Location'].to_dict()
locations = list(set(room_location.values()))
mwf_days = ['Monday', 'Wednesday', 'Friday']
tth_days = ['Tuesday', 'Thursday']
reserve_pct = 0.1

# Preference dictionaries
p1 = pref_lect_course_df.set_index(['Lecturer Name', 'Course Code'])['Preference'].to_dict()
p2 = pref_lect_day_df.set_index(['Lecturer Name', 'Day'])['Preference'].to_dict()
p3 = pref_course_day_df.set_index(['Course Code', 'Day'])['Preference'].to_dict()
p4 = pref_lect_loc_df.set_index(['Lecturer Name', 'Location'])['Preference'].to_dict()
p5 = pref_course_room_df.set_index(['Course Code', 'Room'])['Preference'].to_dict()
p6 = rooms_df.set_index('Room Number')['Room Condition'].map({'Excellent': 2, 'Good': 1, 'Minor Repairs Needed': 0, 'Major Repairs Needed': -1}).to_dict()

# Decision Variables
Z = {}
for l in lecturers:
    for cs in course_sections:
        if a1[l, cs]:
            for d in days:
                for r in rooms:
                    if a4[cs, r] and a5[cs, r] and a7[r]:
                        for t in time_periods:
                            Z[l, cs, d, r, t] = solver.BoolVar(f'Z[{l},{cs},{d},{r},{t}]')

Pattern = {(cs, p): solver.BoolVar(f'Pattern[{cs},{p}]') 
           for cs in course_sections if freq_d[cs] > 1 for p in [0, 1]}
FixedTime = {(cs, t): solver.BoolVar(f'FixedTime[{cs},{t}]') 
             for cs in course_sections if freq_d[cs] > 1 for t in time_periods}
uses_location = {(l, d, loc): solver.BoolVar(f'uses_location[{l},{d},{loc}]') 
                 for l in lecturers for d in days for loc in locations}

# Constraints

# 1. Course Section Frequency and Day Patterns
for cs in course_sections:
    solver.Add(sum(Z.get((l, cs, d, r, t), 0) for l in lecturers for d in days 
                   for r in rooms for t in time_periods) == freq_d[cs])
    if freq_d[cs] > 1:
        solver.Add(Pattern[cs, 0] + Pattern[cs, 1] == 1)
        solver.Add(sum(Z.get((l, cs, d, r, t), 0) for l in lecturers for d in mwf_days 
                       for r in rooms for t in time_periods) == freq_d[cs] * Pattern[cs, 0])
        solver.Add(sum(Z.get((l, cs, d, r, t), 0) for l in lecturers for d in tth_days 
                       for r in rooms for t in time_periods) == freq_d[cs] * Pattern[cs, 1])

# 2. Consistent Time Period
for cs in course_sections:
    if freq_d[cs] > 1:
        solver.Add(sum(FixedTime[cs, t] for t in time_periods) == 1)
        for l in lecturers:
            for d in days:
                for r in rooms:
                    for t in time_periods:
                        solver.Add(Z.get((l, cs, d, r, t), 0) <= FixedTime[cs, t])

# 3. No Lecturer Overlap
for l in lecturers:
    for d in days:
        for t in time_periods:
            solver.Add(sum(Z.get((l, cs, d, r, t), 0) for cs in course_sections for r in rooms) <= 1)

# 4. No Room Overlap
for r in rooms:
    for d in days:
        for t in time_periods:
            solver.Add(sum(Z.get((l, cs, d, r, t), 0) for l in lecturers for cs in course_sections) <= 1)

# 5. Lecturer Availability
for l in lecturers:
    solver.Add(sum(Z.get((l, cs, d, r, t), 0) * dur_t[cs] for cs in course_sections 
                   for d in days for r in rooms for t in time_periods) <= max_t[l])

# 6. Lecturer Breaks
for l in lecturers:
    if break_t[l] == 1:
        for d in days:
            for t_idx in range(len(time_periods) - 1):
                t1, t2 = time_periods[t_idx], time_periods[t_idx + 1]
                solver.Add(sum(Z.get((l, cs1, d, r1, t1), 0) for cs1 in course_sections for r1 in rooms) +
                           sum(Z.get((l, cs2, d, r2, t2), 0) for cs2 in course_sections for r2 in rooms) <= 1)

# 7. Workload Limits
for l in lecturers:
    teaches = solver.NumVar(0, solver.infinity(), f'teaches[{l}]')
    for cs in course_sections:
        total_assign = sum(Z.get((l, cs, d, r, t), 0) for d in days for r in rooms for t in time_periods)
        solver.Add(teaches >= total_assign / freq_d[cs])
    solver.Add(teaches >= min_c[l])
    solver.Add(teaches <= max_c[l])

# 8. Lecturer Location per Day
for l in lecturers:
    for d in days:
        solver.Add(sum(uses_location[l, d, loc] for loc in locations) == 1)
        for loc in locations:
            loc_usage = sum(Z.get((l, cs, d, r, t), 0) for cs in course_sections 
                            for r in rooms if room_location[r] == loc for t in time_periods)
            solver.Add(loc_usage <= uses_location[l, d, loc] * len(course_sections) * len(time_periods))

# 9. Contingency Reservation
total_capacity = len(rooms) * len(days) * len(time_periods)
solver.Add(sum(Z.get((l, cs, d, r, t), 0) for l in lecturers for cs in course_sections 
               for d in days for r in rooms for t in time_periods) <= (1 - reserve_pct) * total_capacity)

# 10. Building Transfer Break
for l in lecturers:
    for d in days:
        for t_idx in range(len(time_periods) - 1):
            t1, t2 = time_periods[t_idx], time_periods[t_idx + 1]
            for r1, r2 in [(r1, r2) for r1 in rooms for r2 in rooms 
                           if room_location[r1] == room_location[r2] and room_building[r1] != room_building[r2]]:
                solver.Add(sum(Z.get((l, cs1, d, r1, t1), 0) for cs1 in course_sections) +
                           sum(Z.get((l, cs2, d, r2, t2), 0) for cs2 in course_sections) <= 1)

# 11. Single Lecturer per Course Section
for cs in course_sections:
    solver.Add(sum(sum(Z.get((l, cs, d, r, t), 0) for d in days for r in rooms for t in time_periods) 
                   for l in lecturers) == freq_d[cs])  # Total assignments match frequency
    for l in lecturers:
        assign = solver.BoolVar(f'assign[{l},{cs}]')
        total_assign = sum(Z.get((l, cs, d, r, t), 0) for d in days for r in rooms for t in time_periods)
        solver.Add(total_assign <= freq_d[cs] * assign)  # If assigned, must match frequency
        solver.Add(total_assign >= assign)  # If any assignment, activate assign
    solver.Add(sum(solver.BoolVar(f'assign[{l},{cs}]') for l in lecturers) <= 1)  # At most one lecturer

# 12. One Session per Day for Multi-Session Courses
for cs in course_sections:
    for d in days:
        solver.Add(sum(Z.get((l, cs, d, r, t), 0) for l in lecturers for r in rooms for t in time_periods) <= 1)

# Objective
objective = solver.Sum(
    Z[l, cs, d, r, t] * (
        p1.get((l, cs.split('-')[0]), 1) +
        p2.get((l, d), 1) +
        p3.get((cs.split('-')[0], d), 1) +
        p4.get((l, room_location[r]), 1) +
        p5.get((cs.split('-')[0], r), 1) +
        p6.get(r, 1)
    ) for l, cs, d, r, t in Z.keys()
)
solver.Maximize(objective)

## Step 3: Solve the Model

In [7]:
# Solve
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print("Solution found!")
    
    # Timetable
    timetable = []
    for l, cs, d, r, t in Z.keys():
        if Z[l, cs, d, r, t].solution_value() > 0.5:
            course = cs.split('-')[0]
            timetable.append({
                'Course': course,
                'Course Section': cs,
                'Lecturer': l,
                'Room': r,
                'Building': room_building[r],
                'Location': room_location[r],
                'Day': d,
                'Time Period': t
            })
    timetable_df = pd.DataFrame(timetable)
    timetable_df = timetable_df[['Course', 'Course Section', 'Lecturer', 'Room', 'Building', 'Location', 'Day', 'Time Period']]
    timetable_df = timetable_df.sort_values(by=['Course', 'Course Section', 'Day'])
    print("\nTimetable:")
    print(timetable_df.to_string(index=False))
    timetable_df.to_excel('uctp-extended-model-output.xlsx', sheet_name='Timetable', index=False)

    # Course Sections Status
    course_status = []
    for cs in course_sections:
        course = cs.split('-')[0]
        assigned = any(Z.get((l, cs, d, r, t), 0).solution_value() > 0.5 for l, cs_, d, r, t in Z.keys() if cs_ == cs)
        course_status.append({'Course': course, 'Course Section': cs, 'Status': 'Assigned' if assigned else 'Available'})
    course_status_df = pd.DataFrame(course_status)
    course_status_df = course_status_df.sort_values(by='Course Section')
    print("\nCourse Sections Status:")
    print(course_status_df.to_string(index=False))
    with pd.ExcelWriter('uctp-extended-model-output.xlsx', mode='a', engine='openpyxl') as writer:
        course_status_df.to_excel(writer, sheet_name='Course Status', index=False)

    # Lecturers Allocation
    lecturer_info = []
    for l in lecturers:
        assigned_hours = sum(Z.get((l, cs, d, r, t), 0).solution_value() * dur_t[cs] * 0.5 
                             for cs in course_sections for d in days for r in rooms for t in time_periods)
        assigned_courses = len(set(cs for cs in course_sections 
                                   if sum(Z.get((l, cs, d, r, t), 0).solution_value() for d in days for r in rooms for t in time_periods) > 0.5))
        faculty_type = lecturers_df[lecturers_df['Lecturer Name'] == l]['Faculty Type'].iloc[0]
        lecturer_info.append({
            'Lecturer': l,
            'Part-Time/Full-Time': faculty_type,
            'Allocated Hours': assigned_hours,
            'Min Course': min_c[l],
            'Max Course': max_c[l],
            'Count of Assigned Course Sections': assigned_courses
        })
    lecturer_info_df = pd.DataFrame(lecturer_info)
    print("\nLecturers Allocation:")
    print(lecturer_info_df.to_string(index=False))
    with pd.ExcelWriter('uctp-extended-model-output.xlsx', mode='a', engine='openpyxl') as writer:
        lecturer_info_df.to_excel(writer, sheet_name='Lecturer Allocation', index=False)

    # Unallocated Rooms and Time Periods
    all_slots = [(r, d, t) for r in rooms for d in days for t in time_periods]
    unallocated = []
    for r, d, t in all_slots:
        if sum(Z.get((l, cs, d, r, t), 0).solution_value() for l in lecturers for cs in course_sections) < 0.5:
            unallocated.append({
                'Room': r,
                'Building': room_building[r],
                'Location': room_location[r],
                'Day': d,
                'Time Period': t
            })
    unallocated_df = pd.DataFrame(unallocated)
    unallocated_df = unallocated_df.sort_values(by='Room')
    print("\nUnallocated Rooms and Time Periods:")
    print(unallocated_df.to_string(index=False))
    with pd.ExcelWriter('uctp-extended-model-output.xlsx', mode='a', engine='openpyxl') as writer:
        unallocated_df.to_excel(writer, sheet_name='Unallocated Slots', index=False)

    print("\nOutput saved to 'uctp-extended-model-output.xlsx'")
else:
    print(f"CP-SAT failed to find a solution. Status: {solver.StatusName()}")
    
    diagnostic_data = {
        "Status": [solver.StatusName()],
        "Conflicts": [solver.NumConflicts()],
        "Branches": [solver.NumBranches()],
        "Wall Time (s)": [solver.WallTime()]
    }
    diagnostic_df = pd.DataFrame(diagnostic_data)
    diagnostic_df.to_excel(output_file, sheet_name="Diagnostics", index=False)
    print(f"\nDiagnostic information saved to {output_file}")

print(f"Time taken: {solver.WallTime()} ms")

Solution found!

Timetable:
Course Course Section          Lecturer    Room          Building Location       Day Time Period
  CPSC     CPSC-500-1   Marin Vratonjic UNF 402 UNF NF Building 2   UNF NF    Friday 11:00-11:30
  CPSC     CPSC-500-1   Marin Vratonjic UNF 215 UNF NF Building 1   UNF NF    Monday 11:00-11:30
  CPSC     CPSC-500-1     Zeeshan Ahmad UNF 215 UNF NF Building 1   UNF NF Wednesday 11:00-11:30
  CPSC     CPSC-500-2        Hany Osman UNF 215 UNF NF Building 1   UNF NF    Friday 16:30-17:00
  CPSC     CPSC-500-2        Hany Osman UNF 215 UNF NF Building 1   UNF NF    Monday 16:30-17:00
  CPSC     CPSC-500-2 Touraj Banirostam UNF 402 UNF NF Building 2   UNF NF Wednesday 16:30-17:00
  CPSC     CPSC-500-3 Hassan Baz Chamas UNF 402 UNF NF Building 2   UNF NF    Friday 16:30-17:00
  CPSC     CPSC-500-3    Ahmed Eltahawi UNF 402 UNF NF Building 2   UNF NF    Monday 16:30-17:00
  CPSC     CPSC-500-3    Ahmed Eltahawi UNF 215 UNF NF Building 1   UNF NF Wednesday 16:30-17:00
  